In [13]:
import numpy as np
from osgeo import gdal
from osgeo import ogr
from skimage import exposure
from skimage.segmentation import quickshift
import geopandas as gpd
import numpy as np
import time
import scipy

In [14]:
naip_fn = 'D:/rambabu_r@nrsc.gov.in/STUDY/MTECH/IIST GeoInformatics Text books/1 SEM/RS/Labs/Object based classifi/Object_Class_segment_1.tiff'

In [15]:
driverTiff = gdal.GetDriverByName('GTiff')
naip_ds = gdal.Open(naip_fn)
nbands = naip_ds.RasterCount
band_data = []

for i in range(1, nbands+1):
    band = naip_ds.GetRasterBand(i).ReadAsArray()
    band_data.append(band)
band_data = np.dstack(band_data)
img = exposure.rescale_intensity(band_data)

In [16]:
from skimage.segmentation import slic

In [17]:
seg_start = time.time()
# do segmentation, different options with quickshift and slic (only use one of the next two lines)
segments = quickshift(img, ratio=0.99, max_dist=5, convert2lab=False)
segments = slic(img, n_segments=500000, compactness=0.1)
print('segments complete', time.time() - seg_start)
 
# save segments to raster
segments_fn = 'D:/rambabu_r@nrsc.gov.in/STUDY/MTECH/IIST GeoInformatics Text books/1 SEM/RS/Labs/Object based classifi/segments.tif'
segments_ds = driverTiff.Create(segments_fn, naip_ds.RasterXSize, naip_ds.RasterYSize,
                                1, gdal.GDT_Float32)
segments_ds.SetGeoTransform(naip_ds.GetGeoTransform())
segments_ds.SetProjection(naip_ds.GetProjectionRef())
segments_ds.GetRasterBand(1).WriteArray(segments)
segments_ds = None

segments complete 0.5018103122711182


In [18]:
def segment_features(segment_pixels):
    features = []
    npixels, nbands = segment_pixels.shape
    for b in range(nbands):
        stats = scipy.stats.describe(segment_pixels[:, b])
        band_stats = list(stats.minmax) + list(stats)[2:]
        if npixels == 1:
            # in this case the variance = nan, change it 0.0
            band_stats[3] = 0.0
        features += band_stats
    return features

In [19]:
segment_ids = np.unique(segments)
objects = []
object_ids = []
for id in segment_ids:
    segment_pixels = img[segments == id]
    object_features = segment_features(segment_pixels)
    objects.append(object_features)
    object_ids.append(id)